In [ ]:
if "google.colab" in str(get_ipython()):
    from google.colab import drive
    import os
    drive.mount("/content/gdrive")
    os.chdir("/content/gdrive/My Drive/CS4243 Project/temp")
    !pwd
    %pip install -q ipywidgets
    %pip install -q -r requirements_colab.txt
else:
    %pip install -q -r requirements.txt

# may need to restart the kernel after installing new packages

# Improved Model (EfficientNet-B0)

In [ ]:
%reset

import torch
import torch.nn as nn

import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from utils import load_tensor

train_x, train_y = load_tensor("unaug", "128", device)
test_x, test_y = load_tensor("test", "128", device)

train_x.size(), train_y.size()

In [ ]:
from utils import render_2d

idx = np.random.choice(train_x.size(0))
render_2d(train_x.cpu()[idx])

## Build CNN

A baseline EfficientNet architecture

In [ ]:
from torchvision import models, transforms

class EfficientNet(nn.Module):
    def __init__(self):
        super(EfficientNet, self).__init__()
        self.layers = nn.Sequential(
            # see https://pytorch.org/vision/stable/models.html for normalisation
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            models.efficientnet_b0(pretrained=True).train(),
            nn.ReLU(),
            nn.Linear(1000, 10)
        )
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
from utils import count_num_params

net = EfficientNet()
print(f"{count_num_params(net):,}")
net

### Training

In [ ]:
from utils import plot_experiments, run_experiments, save_results

def init_func():
    net = EfficientNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(net.parameters(), lr=1e-4)

    return (net, criterion, optimizer)

df_unaug = run_experiments(init_func, (train_x, train_y), (test_x, test_y), (3, 64, 64))
save_results(df_unaug, "efficientnet", "unaug", "64")

In [ ]:
plot_experiments(df_unaug)

In [ ]:
from utils import load_checkpoint, eval_test_accuracy, show_confusion_matrix, show_eval_metrics

net = EfficientNet()
load_checkpoint(net, "35894f38") # to be manually updated

accuracy, pred_y, true_y = eval_test_accuracy(net, (test_x.cpu(), test_y.cpu()), (3, 64, 64))

show_confusion_matrix(pred_y, true_y)
show_eval_metrics(pred_y, true_y)

### Training (with augmented dataset)

In [ ]:
from utils import load_tensor

train_x, train_y = load_tensor("balan", "64", device)
train_x.size(), train_y.size()

In [ ]:
from utils import render_2d

idx = np.random.choice(train_x.size(0))
render_2d(train_x.cpu()[idx])

In [ ]:
from utils import plot_experiments, run_experiments, save_results

def init_func():
    net = EfficientNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(net.parameters(), lr=1e-4)

    return (net, criterion, optimizer)

df_balan = run_experiments(init_func, (train_x, train_y), (test_x, test_y), (3, 64, 64))
save_results(df_balan, "efficientnet", "balan", "64")

In [ ]:
plot_experiments(df_balan)

In [ ]:
from utils import load_checkpoint, eval_test_accuracy, show_confusion_matrix, show_eval_metrics

net = EfficientNet()
load_checkpoint(net, "35894f38") # to be manually updated

accuracy, pred_y, true_y = eval_test_accuracy(net, (test_x.cpu(), test_y.cpu()), (3, 64, 64))

show_confusion_matrix(pred_y, true_y)
show_eval_metrics(pred_y, true_y)